In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [6]:
opts = Options()
opts.headless = True
driver = webdriver.Chrome('./chromedriver',options=opts)

url = 'https://www.edx.org/search?tab=course'

data = []
driver.get(url)
time.sleep(10)
num_pages = int(driver.find_element_by_xpath('//*[@id="displayed-results"]/div[1]/div/nav/ul/li[6]/button').text)
print('Total page: ' + str(num_pages))
for j in range(num_pages):
    print('Page: ' + str(j + 1))
    if j > 0:
        driver.find_element_by_xpath("//*[@id=\"displayed-results\"]/div[1]/div/nav/ul/li[7]/button").click()
        time.sleep(5)
    else:
        time.sleep(0)

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "html.parser")
    summary = soup.find("div",{'static-card-list d-flex m-xl-0 flex-wrap'})
    for i in summary:
        link = [u['href'] for u in i.find_all('a', href=True)]
        data.append([link])
    df = pd.DataFrame(data, columns=['link'])

for i in range(len(df['link'])):
    df['link'][i] = str(df['link'][i]).replace('[','').replace("'",'').replace(']','')
    df['link'][i] = 'https://www.edx.org' + str(df['link'][i])
# df.to_csv('edx_links.csv', sep = ',',encoding='utf-8-sig')
df.head()

Total page: 42
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41
Page: 42


,link
0,https://www.edx.org/course/how-to-learn-online
1,https://www.edx.org/course/programming-for-eve...
2,https://www.edx.org/course/cs50s-introduction-...
3,https://www.edx.org/course/introduction-to-pro...
4,https://www.edx.org/course/marketing-analytics...


In [7]:
# linksdf = pd.read_csv("./edx_links.csv", sep=",")
linksdf = df

url = ''
urls = []

# num_pages = len(linksdf['link'])
num_pages = 3
for i in range(num_pages):
    url_page = linksdf['link'][i]
    urls.append(url_page)

data = []
for url in urls:
    print(url)
    driver.get(url)
    time.sleep(5)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "html.parser")

    title = soup.title.text
    author = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[4]/div[2]/a").text
    desc = soup.find('div', {'class': 'course-description'}).findNext('p').text
    subject = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[5]/div[2]/a").text
    price = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[3]/div[2]/p/span[1]").text
    img_link = [u['src'] for u in soup.find_all('img', src=True)][1]
    data.append([title, author, price, subject, desc, img_link])
    
    time.sleep(1)

df = pd.DataFrame(data, columns=['title', 'author', 'price', 'subject', 'desc', 'img_link'])
df['title'] = df['title'].str.replace("edX","")
df['title'] = df['title'].str.replace("|","")
df['price'] = df['price'].str.replace("FREE","0")
df.to_csv('edx_courses.csv', sep = ',',encoding='utf-8-sig')
df.head()

https://www.edx.org/course/how-to-learn-online
https://www.edx.org/course/programming-for-everybody-getting-started-with-pyt
https://www.edx.org/course/cs50s-introduction-to-computer-science


,title,author,price,subject,desc,img_link
0,How to Learn Online,edX,0,Education & Teacher Training,"Designed for those who are new to elearning, t...",https://prod-discovery.edx-cdn.org/organizatio...
1,Programming for Everybody (Getting Started wit...,MichiganX,0,Computer Science,This course aims to teach everyone the basics ...,https://prod-discovery.edx-cdn.org/organizatio...
2,CS50's Introduction to Computer Science,HarvardX,0,Computer Science,"This is CS50x , Harvard University's introduct...",https://prod-discovery.edx-cdn.org/organizatio...
